***Connect to google drive colab***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***Installation of dependencies***

In [ ]:
!pip install assemblyai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.6/72.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [ ]:
!pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.6 MB/s eta 0:00:00


***Display audio***

In [ ]:
from IPython.display import Audio

# Function to handle audio playback
def play_audio(file_path, rate=None):
    """Plays an audio file.

    Args:
        file_path (str): The path to the audio file.
        rate (int, optional): The sample rate of the audio.
                               If None, attempts to autodetect.
                               Defaults to None.
    """
    # If rate is not provided, assume it's an audio file and let Audio autodetect
    # If rate is provided, assume data is numpy array and pass rate
    audio = Audio(filename=file_path, rate=44100)
    display(audio)


audio_file = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /Custom audios + transcriptions/audio/S1.m4a"
play_audio(audio_file)

***Speech recognation***

In [ ]:
import requests
import json
import time

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

def upload_file(api_token, path):
    """
    Upload a file to the AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        path (str): Path to the local file.

    Returns:
        str: The upload URL.
    """
    headers = {'authorization': api_token}
    response = requests.post('https://api.assemblyai.com/v2/upload',
                             headers=headers,
                             data=read_file(path))

    if response.status_code == 200:
        return response.json()["upload_url"]
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def create_transcript(api_token, audio_url):
    """
    Create a transcript using AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        audio_url (str): URL of the audio file to be transcribed.

    Returns:
        dict: Completed transcript object.
    """
    # Set the API endpoint for creating a new transcript
    url = "https://api.assemblyai.com/v2/transcript"

    # Set the headers for the request, including the API token and content type
    headers = {
        "authorization": api_token,
        "content-type": "application/json"
    }

    # Set the data for the request, including the URL of the audio file to be transcribed
    data = {
        "audio_url": audio_url
    }

    # Send a POST request to the API to create a new transcript, passing in the headers and data
    response = requests.post(url, json=data, headers=headers)

    # Get the transcript ID from the response JSON data
    transcript_id = response.json()['id']

    # Set the polling endpoint URL by appending the transcript ID to the API endpoint
    polling_endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    # Keep polling the API until the transcription is complete
    while True:
        # Send a GET request to the polling endpoint, passing in the headers
        transcription_result = requests.get(polling_endpoint, headers=headers).json()

        # If the status of the transcription is 'completed', exit the loop
        if transcription_result['status'] == 'completed':
            break

        # If the status of the transcription is 'error', raise a runtime error with the error message
        elif transcription_result['status'] == 'error':
            raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

        # If the status of the transcription is not 'completed' or 'error', wait for 3 seconds and poll again
        else:
            time.sleep(3)

    return transcription_result

your_api_token = "bc7e213c3d14435d9f1e198ce37badb2"

# Upload a local file
filename = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /Custom audios + transcriptions/audio/S1.m4a"
upload_url = upload_file(your_api_token, filename)

# Transcribe it
transcript = create_transcript(your_api_token, upload_url)

# Print the completed transcript object
result=transcript['text']
print(result)

Good morning. What brings you in today? Good morning, Doctor. I have been experiencing some lower back pain for the past couple of weeks. I'm sorry to hear that. Can you describe the pain? Is it sharp, dull or more of an ache? It's more of a dull ache, but it gets worse when I sit for long periods or bend over. Understood. On a scale of 1 to 10, how would you rate the pain when it's at its worst? I did say it's about a seven when it's bad. That sounds uncomfortable. Have you tried anything to relieve the pain like ice, rest or over the counter medication? I have tried resting and taking some ibuprofen, which helps a bit, but it always comes back when I move around. I see. Have you had any previous back issues or did this start suddenly? I had some minor issues a few years ago, but nothing like this. Got it. Let's do a physical examination to assess your range of motion and check any tenderness. After that, we can discuss the potential treatment.


In [ ]:
print(transcript)

{'id': '8152cd1a-c388-4e43-b200-6475e19538db', 'language_model': 'assemblyai_default', 'acoustic_model': 'assemblyai_default', 'language_code': 'en_us', 'status': 'completed', 'audio_url': 'https://cdn.assemblyai.com/upload/72746c91-cba5-48e0-983f-2ce221e869dc', 'text': "Good morning. What brings you in today? Good morning, Doctor. I have been experiencing some lower back pain for the past couple of weeks. I'm sorry to hear that. Can you describe the pain? Is it sharp, dull or more of an ache? It's more of a dull ache, but it gets worse when I sit for long periods or bend over. Understood. On a scale of 1 to 10, how would you rate the pain when it's at its worst? I did say it's about a seven when it's bad. That sounds uncomfortable. Have you tried anything to relieve the pain like ice, rest or over the counter medication? I have tried resting and taking some ibuprofen, which helps a bit, but it always comes back when I move around. I see. Have you had any previous back issues or did th

'audio_url': 'https://cdn.assemblyai.com/upload/35c6cb08-b9bd-481e-9682-3cbbb0f67397'

In [ ]:
# Split the transcript into chunks of maximum length 80 characters
chunks = [transcript['text'][i:i+110] for i in range(0, len(transcript['text']), 110)]

# Join the chunks with line breaks
formatted_text = "\n".join(chunks)

print(formatted_text)

Good morning. What brings you in today? Good morning, Doctor. I have been experiencing some lower back pain fo
r the past couple of weeks. I'm sorry to hear that. Can you describe the pain? Is it sharp, dull or more of an
 ache? It's more of a dull ache, but it gets worse when I sit for long periods or bend over. Understood. On a 
scale of 1 to 10, how would you rate the pain when it's at its worst? I did say it's about a seven when it's b
ad. That sounds uncomfortable. Have you tried anything to relieve the pain like ice, rest or over the counter 
medication? I have tried resting and taking some ibuprofen, which helps a bit, but it always comes back when I
 move around. I see. Have you had any previous back issues or did this start suddenly? I had some minor issues
 a few years ago, but nothing like this. Got it. Let's do a physical examination to assess your range of motio
n and check any tenderness. After that, we can discuss the potential treatment.


***Speaker Diarization***

In [ ]:
import requests
import json
import time

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

def upload_file(api_token, path):
    """
    Upload a file to the AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        path (str): Path to the local file.

    Returns:
        str: The upload URL.
    """
    headers = {'authorization': api_token}
    response = requests.post('https://api.assemblyai.com/v2/upload',
                             headers=headers,
                             data=read_file(path))

    if response.status_code == 200:
        return response.json()["upload_url"]
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def create_transcript(api_token, audio_url):
    """
    Create a transcript using AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        audio_url (str): URL of the audio file to be transcribed.

    Returns:
        dict: Completed transcript object.
    """
    # Set the API endpoint for creating a new transcript
    url = "https://api.assemblyai.com/v2/transcript"

    # Set the headers for the request, including the API token and content type
    headers = {
        "authorization": api_token,
        "content-type": "application/json"
    }

    # Set the data for the request, including the URL of the audio file to be transcribed
    data = {
        "audio_url": audio_url,
        "speaker_labels": True
    }

    # Send a POST request to the API to create a new transcript, passing in the headers and data
    response = requests.post(url, json=data, headers=headers)

    # Get the transcript ID from the response JSON data
    transcript_id = response.json()['id']

    # Set the polling endpoint URL by appending the transcript ID to the API endpoint
    polling_endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    # Keep polling the API until the transcription is complete
    while True:
        # Send a GET request to the polling endpoint, passing in the headers
        transcription_result = requests.get(polling_endpoint, headers=headers).json()

        # If the status of the transcription is 'completed', exit the loop
        if transcription_result['status'] == 'completed':
            break

        # If the status of the transcription is 'error', raise a runtime error with the error message
        elif transcription_result['status'] == 'error':
            raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

        # If the status of the transcription is not 'completed' or 'error', wait for 3 seconds and poll again
        else:
            time.sleep(3)

    return transcription_result

your_api_token = "bc7e213c3d14435d9f1e198ce37badb2"

# Upload a local file
filename = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /Custom audios + transcriptions/audio/S1.m4a"
upload_url = upload_file(your_api_token, filename)

# Transcribe it
transcrSpeaker = create_transcript(your_api_token, upload_url)

# Iterate through the utterances and print speaker information
print("\nSpeakers' Information:\n\n")

for utterance in transcrSpeaker['utterances']:
    speaker = utterance['speaker']
    text = utterance['text']
    confidence = utterance['confidence']

    print(f"Speaker {speaker}: {text}\nConfidence: {confidence}\n")


Speakers' Information:


Speaker A: Good morning. What brings you in today?
Confidence: 0.99617

Speaker B: Good morning, Doctor. I have been experiencing some lower back pain for the past couple of weeks.
Confidence: 0.9216176

Speaker A: I'm sorry to hear that. Can you describe the pain? Is it sharp, dull or more of an ache?
Confidence: 0.9214047

Speaker B: It's more of a dull ache, but it gets worse when I sit for long periods or bend over.
Confidence: 0.93219316

Speaker A: Understood. On a scale of 1 to 10, how would you rate the pain when it's at its worst?
Confidence: 0.91038895

Speaker B: I did say it's about a seven when it's bad.
Confidence: 0.888983

Speaker A: That sounds uncomfortable. Have you tried anything to relieve the pain like ice, rest or over the counter medication?
Confidence: 0.87183946

Speaker B: I have tried resting and taking some ibuprofen, which helps a bit, but it always comes back when I move around.
Confidence: 0.94818

Speaker A: I see. Have you had

In [ ]:
print(transcrSpeaker['utterances'])

[{'speaker': 'A', 'text': 'Good morning. What brings you in today?', 'confidence': 0.99617, 'start': 640, 'end': 2925, 'words': [{'text': 'Good', 'start': 640, 'end': 776, 'confidence': 0.99522, 'speaker': 'A'}, {'text': 'morning.', 'start': 776, 'end': 1208, 'confidence': 0.99982, 'speaker': 'A'}, {'text': 'What', 'start': 1225, 'end': 1481, 'confidence': 0.99937, 'speaker': 'A'}, {'text': 'brings', 'start': 1513, 'end': 1777, 'confidence': 0.99417, 'speaker': 'A'}, {'text': 'you', 'start': 1801, 'end': 1985, 'confidence': 0.99632, 'speaker': 'A'}, {'text': 'in', 'start': 2025, 'end': 2273, 'confidence': 0.99003, 'speaker': 'A'}, {'text': 'today?', 'start': 2329, 'end': 2925, 'confidence': 0.99826, 'speaker': 'A'}]}, {'speaker': 'B', 'text': 'Good morning, Doctor. I have been experiencing some lower back pain for the past couple of weeks.', 'confidence': 0.9216176, 'start': 3345, 'end': 8129, 'words': [{'text': 'Good', 'start': 3345, 'end': 3657, 'confidence': 0.99749, 'speaker': 'B'}

***Summarization***

In [ ]:
import requests
import json
import time

def read_file(filename, chunk_size=5242880):
    with open(filename, 'rb') as _file:
        while True:
            data = _file.read(chunk_size)
            if not data:
                break
            yield data

def upload_file(api_token, path):
    """
    Upload a file to the AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        path (str): Path to the local file.

    Returns:
        str: The upload URL.
    """
    headers = {'authorization': api_token}
    response = requests.post('https://api.assemblyai.com/v2/upload',
                             headers=headers,
                             data=read_file(path))

    if response.status_code == 200:
        return response.json()["upload_url"]
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def create_transcript(api_token, audio_url):
    """
    Create a transcript using AssemblyAI API.

    Args:
        api_token (str): Your API token for AssemblyAI.
        audio_url (str): URL of the audio file to be transcribed.

    Returns:
        dict: Completed transcript object.
    """
    # Set the API endpoint for creating a new transcript
    url = "https://api.assemblyai.com/v2/transcript"

    # Set the headers for the request, including the API token and content type
    headers = {
        "authorization": api_token,
        "content-type": "application/json"
    }

    # Set the data for the request, including the URL of the audio file to be transcribed
    data = {
        "audio_url": audio_url,
        "summarization": True,
        "summary_model": "informative",
        "summary_type": "bullets"
    }

    # Send a POST request to the API to create a new transcript, passing in the headers and data
    response = requests.post(url, json=data, headers=headers)

    # Get the transcript ID from the response JSON data
    transcript_id = response.json()['id']

    # Set the polling endpoint URL by appending the transcript ID to the API endpoint
    polling_endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    # Keep polling the API until the transcription is complete
    while True:
        # Send a GET request to the polling endpoint, passing in the headers
        transcription_result = requests.get(polling_endpoint, headers=headers).json()

        # If the status of the transcription is 'completed', exit the loop
        if transcription_result['status'] == 'completed':
            break

        # If the status of the transcription is 'error', raise a runtime error with the error message
        elif transcription_result['status'] == 'error':
            raise RuntimeError(f"Transcription failed: {transcription_result['error']}")

        # If the status of the transcription is not 'completed' or 'error', wait for 3 seconds and poll again
        else:
            time.sleep(3)

    return transcription_result

your_api_token = "bc7e213c3d14435d9f1e198ce37badb2"

# Upload a local file
filename = "/content/drive/MyDrive/AI_ML_INTERN /Task 3 Practice /Custom audios + transcriptions/audio/S1.m4a"
upload_url = upload_file(your_api_token, filename)

# Transcribe it
transcript = create_transcript(your_api_token, upload_url)

# Print the summary
print(transcript["summary"])

- Lower back pain gets worse when I sit for long periods or bend over. Have you tried anything to relieve the pain like ice, rest or over the counter medication? Let's do a physical examination to assess your range of motion and check any tenderness. After that, we can discuss the potential treatment.


***Translate the text from english to any language the user want***

In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='fr')

# Split the text into smaller chunks
chunk_size = 500  # Adjust as per your needs
chunks = [result[i:i+chunk_size] for i in range(0, len(result), chunk_size)]

# Translate each chunk and store the results
translations = []
for chunk in chunks:
    translation = translator.translate(chunk)
    translations.append(translation)

# Concatenate the translations
translated_text = " ".join(translations)

print(translated_text)

Bonjour. Qu'est-ce qui vous amène aujourd'hui ? Bonjour, Docteur. J'ai des douleurs dans le bas du dos depuis quelques semaines. Je suis désolé de l'entendre. Pouvez-vous décrire la douleur ? Est-elle vive, sourde ou plus douloureuse ? C'est plutôt une douleur sourde, mais elle s'aggrave lorsque je reste assis pendant de longues périodes ou que je me penche. Compris. Sur une échelle de 1 à 10, comment évalueriez-vous la douleur lorsqu'elle est à son apogée ? J'ai dit qu'elle était d'environ sept lorsqu'elle est intense. Cela semble inconfortable. Avez-vous essayé quelque chose pour soulager la douleur ? Est-ce que tu ressens la douleur comme de la glace, du repos ou des médicaments en vente libre ? J'ai essayé de me reposer et de prendre de l'ibuprofène, ce qui m'aide un peu, mais la douleur revient toujours quand je bouge. Je vois. As-tu déjà eu des problèmes de dos ou est-ce que cela a commencé soudainement ? J'ai eu quelques problèmes mineurs il y a quelques années, mais rien de tel

In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='de')


# Split the text into smaller chunks
chunk_size = 500  # Adjust as per your needs
chunks = [transcript["summary"][i:i+chunk_size] for i in range(0, len(transcript["summary"]), chunk_size)]

# Translate each chunk and store the results
translations = []
for chunk in chunks:
    translation = translator.translate(chunk)
    translations.append(translation)

# Concatenate the translations
translated_text = " ".join(translations)

print(translated_text)



- Die Schmerzen im unteren Rückenbereich werden schlimmer, wenn ich lange sitze oder mich vorbeuge. Haben Sie schon versucht, die Schmerzen zu lindern, z. B. mit Eis, Ruhe oder rezeptfreien Medikamenten? Lassen Sie uns eine körperliche Untersuchung durchführen, um Ihren Bewegungsbereich zu beurteilen und eventuelle Druckempfindlichkeiten festzustellen. Danach können wir die mögliche Behandlung besprechen.


In [ ]:
# Split the transcript into chunks of maximum length 80 characters
chunks = [translated_text[i:i+110] for i in range(0, len(translated_text), 110)]

# Join the chunks with line breaks
formatted_text_Trans= "\n".join(chunks)

print(formatted_text_Trans)

- Die Schmerzen im unteren Rückenbereich werden schlimmer, wenn ich lange sitze oder mich vorbeuge. Haben Sie 
schon versucht, die Schmerzen zu lindern, z. B. mit Eis, Ruhe oder rezeptfreien Medikamenten? Lassen Sie uns e
ine körperliche Untersuchung durchführen, um Ihren Bewegungsbereich zu beurteilen und eventuelle Druckempfindl
ichkeiten festzustellen. Danach können wir die mögliche Behandlung besprechen.


In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='auto', target='mr')

# Iterate through the utterances and print translated speaker information
print("\nInformation des intervenants:\n\n")

for utterance in transcrSpeaker['utterances']:
    speaker = utterance['speaker']
    text = utterance['text']
    confidence = utterance['confidence']

    # Translate the speaker and text
    translated_speaker = translator.translate(speaker)
    translated_text = translator.translate(text)

    print(f"Intervenant {translated_speaker}: {translated_text}\nConfiance: {confidence}\n")



Information des intervenants:


Intervenant ए: शुभ सकाळ. आज तुम्हाला काय आणले आहे?
Confiance: 0.99617

Intervenant बी: सुप्रभात, डॉक्टर. गेल्या काही आठवड्यांपासून मला पाठीच्या खालच्या भागात काही वेदना होत आहेत.
Confiance: 0.9216176

Intervenant ए: मला ते ऐकून वाईट वाटले. आपण वेदना वर्णन करू शकता? ती तीक्ष्ण, कंटाळवाणा किंवा अधिक वेदना आहे का?
Confiance: 0.9214047

Intervenant बी: हे एक कंटाळवाणे वेदना अधिक आहे, परंतु जेव्हा मी बराच वेळ बसतो किंवा वाकतो तेव्हा ते आणखी वाईट होते.
Confiance: 0.93219316

Intervenant ए: समजले. 1 ते 10 च्या स्केलवर, वेदना सर्वात वाईट असताना तुम्ही त्याचे मूल्यांकन कसे कराल?
Confiance: 0.91038895

Intervenant बी: मी ते वाईट आहे तेव्हा सुमारे सात आहे म्हणालो.
Confiance: 0.888983

Intervenant ए: ते अस्वस्थ वाटतं. बर्फ, विश्रांती किंवा काउंटरवर औषधोपचार यांसारख्या वेदना कमी करण्यासाठी तुम्ही काही प्रयत्न केले आहेत का?
Confiance: 0.87183946

Intervenant बी: मी विश्रांती घेण्याचा आणि काही आयबुप्रोफेन घेण्याचा प्रयत्न केला आहे, ज्यामुळे थोडी मदत होते, परंतु जेव्हा